Creating Reviewer-Recipe-Rating matrix :

In [328]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [329]:
final_processed_dataset=pd.read_csv(r"Final_Processed_Dataset.csv")

reviewer_recipe_rating_matrix = final_processed_dataset.pivot_table(index='ReviewerName', columns='Name', values='Rating')

Normalising rating of each reviewer's rating to penalise ratings of those who ovverrate/underrate irratically:

In [330]:
def normalise_rating(user_ratings):
    return user_ratings/user_ratings.sum()*user_ratings.count()

In [331]:
reviewer_recipe_rating_matrix=reviewer_recipe_rating_matrix.apply(normalise_rating,axis=1)

Calculating cosine similarity matrix :

In [332]:
reviewer_recipe_rating_relation_matrix=reviewer_recipe_rating_matrix.fillna(0)

from scipy.sparse import csr_matrix

reviewer_recipe_rating_relation_matrix=csr_matrix(reviewer_recipe_rating_relation_matrix.values)
cosine_similarity_matrix=cosine_similarity(reviewer_recipe_rating_relation_matrix)
cosine_similarity_matrix = pd.DataFrame(cosine_similarity_matrix, index=reviewer_recipe_rating_matrix.index, columns=reviewer_recipe_rating_matrix.index)

Predicting Ratings for all recipes where rating is nan for a each user one by one :

In [333]:
# user=input("Tell your ReviewerName : ")
user="Bergy"

user_index=cosine_similarity_matrix.index.get_loc(user)
user_similarity=cosine_similarity_matrix[user]
user_similarity.drop(user,inplace=True)
# user_similarity=user_similarity/user_similarity.sum()  ---Wrong. weights.sum() should not include where nan values
user_similarity=user_similarity.reset_index()
reviewer_recipe_rating_matrix_temp=reviewer_recipe_rating_matrix.reset_index()

Removing already rated/eaten dishes from possible recommendations :

In [334]:
user_data=reviewer_recipe_rating_matrix_temp[reviewer_recipe_rating_matrix_temp["ReviewerName"]==user]
user_data=user_data.drop("ReviewerName",axis=1)
already_rated = user_data.loc[:, user_data.notna().any()]
reviewer_recipe_rating_matrix_temp.drop(columns=already_rated.columns.tolist(),inplace=True)

Applying appropropriate weights to ratings by other users as per similarity :

In [335]:
def apply_similarity_weights(row):
    return row*row.loc[user]

In [336]:
merged_data=pd.merge(user_similarity,reviewer_recipe_rating_matrix_temp,on="ReviewerName")
merged_data.drop("ReviewerName",axis=1,inplace=True)
merged_data=merged_data.apply(apply_similarity_weights,axis=1)

Normalising the weights :

In [337]:
def weighted_ratings(column):
    return column/column.count()

In [338]:
merged_data=merged_data.apply(weighted_ratings,axis=0)
merged_data.drop(user,axis=1,inplace=True)

Final Recommendations :

In [339]:
top_20_recommend=merged_data.sum(axis=0).sort_values(ascending=False).iloc[0:20]
print(top_20_recommend.reset_index()["index"])

0                               Ham &amp; Egg Casserole
1     Shrimp Stir-Fry With Bok Choy, Mushrooms & Pep...
2       Perch or Snapper Fillet With Tomatoes and Onion
3             Filet of Sole With Spinach &amp; Tomatoes
4                                        Turtle Cookies
5                          Apple Cider Vinegar Marinade
6                                   Roasted Red Snapper
7                            Healthy Egg Salad Sandwich
8                                 Diabetic Bran Muffins
9                                     Healthy Manicotti
10                               Easy Cherry Cheesecake
11                Camembert Fondue With Truffle Essence
12                                       Rich Cornbread
13                      Easy Microwave Rocky Road Fudge
14                                    Hot Chicken Wings
15    Spinach Walnut Salad W. Raspberry Walnut Vinai...
16                            Ricotta Stuffed Mushrooms
17                               Stuffed Baked P